# Predicting Wine Quality

### Data from: kaggle.com via UCI machine learning repository

Data Source:
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

### Compiled by: Dana Hagist

In this project, I'm going to be looking at the variables that impact wine quality, classifying the wine as "GOOD" or "BAD."

Although the original data includes a 1-10 scale for quality, I am defining a good wine as anything with a 7 or above, and bad as anything below 7.

Obviously, this doesn't mean that all wines that are 6 or 5 are actually bad, but gives a meaningful cutoff for a true logistic regression analysis.

In [31]:
# Set working directory
setwd # ('') Insert working directory between ticks

In [5]:
# Read in the dataset and understand structure and summary statistics
options("scipen"=999)
wine = read.csv("winequality-red.csv")
str(wine)
summary(wine)

'data.frame':	1599 obs. of  14 variables:
 $ fixed.acidity       : num  7.4 7.8 7.8 11.2 7.4 7.4 7.9 7.3 7.8 7.5 ...
 $ volatile.acidity    : num  0.7 0.88 0.76 0.28 0.7 0.66 0.6 0.65 0.58 0.5 ...
 $ citric.acid         : num  0 0 0.04 0.56 0 0 0.06 0 0.02 0.36 ...
 $ residual.sugar      : num  1.9 2.6 2.3 1.9 1.9 1.8 1.6 1.2 2 6.1 ...
 $ chlorides           : num  0.076 0.098 0.092 0.075 0.076 0.075 0.069 0.065 0.073 0.071 ...
 $ free.sulfur.dioxide : num  11 25 15 17 11 13 15 15 9 17 ...
 $ total.sulfur.dioxide: num  34 67 54 60 34 40 59 21 18 102 ...
 $ density             : num  0.998 0.997 0.997 0.998 0.998 ...
 $ pH                  : num  3.51 3.2 3.26 3.16 3.51 3.51 3.3 3.39 3.36 3.35 ...
 $ sulphates           : num  0.56 0.68 0.65 0.58 0.56 0.56 0.46 0.47 0.57 0.8 ...
 $ alcohol             : num  9.4 9.8 9.8 9.8 9.4 9.4 9.4 10 9.5 10.5 ...
 $ quality             : int  5 5 5 6 5 5 5 7 7 5 ...
 $ good                : Factor w/ 2 levels "BAD","GOOD": 1 1 1 1 1 1 1 2 2 1 ...
 

 fixed.acidity   volatile.acidity  citric.acid    residual.sugar  
 Min.   : 4.60   Min.   :0.1200   Min.   :0.000   Min.   : 0.900  
 1st Qu.: 7.10   1st Qu.:0.3900   1st Qu.:0.090   1st Qu.: 1.900  
 Median : 7.90   Median :0.5200   Median :0.260   Median : 2.200  
 Mean   : 8.32   Mean   :0.5278   Mean   :0.271   Mean   : 2.539  
 3rd Qu.: 9.20   3rd Qu.:0.6400   3rd Qu.:0.420   3rd Qu.: 2.600  
 Max.   :15.90   Max.   :1.5800   Max.   :1.000   Max.   :15.500  
   chlorides       free.sulfur.dioxide total.sulfur.dioxide    density      
 Min.   :0.01200   Min.   : 1.00       Min.   :  6.00       Min.   :0.9901  
 1st Qu.:0.07000   1st Qu.: 7.00       1st Qu.: 22.00       1st Qu.:0.9956  
 Median :0.07900   Median :14.00       Median : 38.00       Median :0.9968  
 Mean   :0.08747   Mean   :15.87       Mean   : 46.47       Mean   :0.9967  
 3rd Qu.:0.09000   3rd Qu.:21.00       3rd Qu.: 62.00       3rd Qu.:0.9978  
 Max.   :0.61100   Max.   :72.00       Max.   :289.00       Max.   :1

Based on the above, it does not appear that we have any issues with regards to missing data or other data quality issues.

We can move forward with creating and exploring some initial models.

In [6]:
# Split data into train and test sets
set.seed(144)
library(caTools)
split = sample.split(wine$good_binary, SplitRatio = .7)
train = subset(wine, split == TRUE)
test = subset(wine, split == FALSE)
str(train)
str(test)

'data.frame':	1119 obs. of  14 variables:
 $ fixed.acidity       : num  7.4 11.2 7.4 7.4 7.9 7.8 6.7 5.6 7.8 8.9 ...
 $ volatile.acidity    : num  0.7 0.28 0.7 0.66 0.6 0.58 0.58 0.615 0.61 0.62 ...
 $ citric.acid         : num  0 0.56 0 0 0.06 0.02 0.08 0 0.29 0.18 ...
 $ residual.sugar      : num  1.9 1.9 1.9 1.8 1.6 2 1.8 1.6 1.6 3.8 ...
 $ chlorides           : num  0.076 0.075 0.076 0.075 0.069 0.073 0.097 0.089 0.114 0.176 ...
 $ free.sulfur.dioxide : num  11 17 11 13 15 9 15 16 9 52 ...
 $ total.sulfur.dioxide: num  34 60 34 40 59 18 65 59 29 145 ...
 $ density             : num  0.998 0.998 0.998 0.998 0.996 ...
 $ pH                  : num  3.51 3.16 3.51 3.51 3.3 3.36 3.28 3.58 3.26 3.16 ...
 $ sulphates           : num  0.56 0.58 0.56 0.56 0.46 0.57 0.54 0.52 1.56 0.88 ...
 $ alcohol             : num  9.4 9.8 9.4 9.4 9.4 9.5 9.2 9.9 9.1 9.2 ...
 $ quality             : int  5 6 5 5 5 7 5 5 5 5 ...
 $ good                : Factor w/ 2 levels "BAD","GOOD": 1 1 1 1 1 2 1 1 1 1

In [7]:
# Creating baseline model
table(wine$good_binary)


   0    1 
1382  217 

With the above, a simple baseline model would predict that all of the wines are going to be bad, as there are more in the bad category.

In [63]:
# Understanding level of collinearity
vars = train[c('fixed.acidity', 'volatile.acidity','citric.acid', 'residual.sugar', 'chlorides', 'free.sulfur.dioxide', 'total.sulfur.dioxide', 'density', 'pH', 'sulphates', 'alcohol')]
cor(vars)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol
fixed.acidity,1.00000000,-0.24835951,0.66328052,0.10050922,0.076418084,-0.150099400,-0.11719655,0.67319186,-0.68323493,0.17893957,-0.07620950
volatile.acidity,-0.24835951,1.00000000,-0.54978974,0.01328370,0.066869886,-0.028394844,0.06327700,0.03223019,0.23546850,-0.25096275,-0.20161095
citric.acid,0.66328052,-0.54978974,1.00000000,0.13008315,0.219685566,-0.053819876,0.04022611,0.35824113,-0.54904177,0.31645489,0.09898899
residual.sugar,0.10050922,0.01328370,0.13008315,1.00000000,0.011023345,0.160191746,0.21805630,0.33209398,-0.07754659,-0.02723708,0.04940878
chlorides,0.07641808,0.06686989,0.21968557,0.01102334,1.000000000,-0.001964367,0.03776908,0.17877205,-0.28031260,0.39587721,-0.22851435
free.sulfur.dioxide,-0.15009940,-0.02839484,-0.05381988,0.16019175,-0.001964367,1.000000000,0.65415982,-0.01778014,0.07246603,0.06655326,-0.05928800
total.sulfur.dioxide,-0.11719655,0.06327700,0.04022611,0.21805630,0.037769083,0.654159820,1.00000000,0.06436244,-0.07316460,0.05285608,-0.18224940
density,0.67319186,0.03223019,0.35824113,0.33209398,0.178772046,-0.017780137,0.06436244,1.00000000,-0.34998744,0.13313609,-0.50543414
pH,-0.68323493,0.23546850,-0.54904177,-0.07754659,-0.280312601,0.072466035,-0.07316460,-0.34998744,1.00000000,-0.22784171,0.21652765
sulphates,0.17893957,-0.25096275,0.31645489,-0.02723708,0.395877206,0.066553264,0.05285608,0.13313609,-0.22784171,1.00000000,0.08301249


Looking at the above table of correlations, it does not appear that we will deal with a significant multicollinearity problem, as the correlation between many of these variables is relatively small.

In [64]:
# Building First Model with only one variable, alcohol
mod1 = glm(good_binary ~ alcohol, data=train, family="binomial")
summary(mod1)


Call:
glm(formula = good_binary ~ alcohol, family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3006  -0.4955  -0.3083  -0.2355   2.7225  

Coefficients:
             Estimate Std. Error z value            Pr(>|z|)    
(Intercept) -13.77478    1.02811  -13.40 <0.0000000000000002 ***
alcohol       1.09715    0.09121   12.03 <0.0000000000000002 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 889.23  on 1118  degrees of freedom
Residual deviance: 705.69  on 1117  degrees of freedom
AIC: 709.69

Number of Fisher Scoring iterations: 5


In [65]:
# Computing predictions
pred1 = predict(mod1, type="response")
table(train$good_binary, pred1 > 0.5)

   
    FALSE TRUE
  0   938   29
  1   129   23

In [66]:
# Adding variable(s) to our model
# We will look for variables that are highly UNcorrelated to include.
mod2 = glm(good_binary ~ alcohol + fixed.acidity + residual.sugar + free.sulfur.dioxide + density + sulphates, data=train, family="binomial")
summary(mod2)


Call:
glm(formula = good_binary ~ alcohol + fixed.acidity + residual.sugar + 
    free.sulfur.dioxide + density + sulphates, family = "binomial", 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.6373  -0.4356  -0.2213  -0.1349   2.9674  

Coefficients:
                      Estimate Std. Error z value         Pr(>|z|)    
(Intercept)          406.61533  101.83377   3.993 0.00006526131879 ***
alcohol                0.91511    0.13201   6.932 0.00000000000414 ***
fixed.acidity          0.53529    0.08927   5.996 0.00000000201895 ***
residual.sugar         0.21053    0.08598   2.449           0.0143 *  
free.sulfur.dioxide   -0.02136    0.01109  -1.926           0.0541 .  
density             -427.48508  102.13616  -4.185 0.00002846105169 ***
sulphates              3.81220    0.54146   7.041 0.00000000000191 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null de

In [67]:
# Getting predictions for second model
pred2 = predict(mod2, type="response")
table(train$good_binary, pred2 > 0.5)

   
    FALSE TRUE
  0   936   31
  1   104   48

In [68]:
# Trying another model with all of the variables
mod3 = glm(good_binary ~ alcohol + fixed.acidity + volatile.acidity + citric.acid + chlorides + total.sulfur.dioxide + pH +  residual.sugar + free.sulfur.dioxide + density + sulphates, data=train, family="binomial")
summary(mod3)


Call:
glm(formula = good_binary ~ alcohol + fixed.acidity + volatile.acidity + 
    citric.acid + chlorides + total.sulfur.dioxide + pH + residual.sugar + 
    free.sulfur.dioxide + density + sulphates, family = "binomial", 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.2033  -0.4052  -0.1845  -0.0988   3.1090  

Coefficients:
                        Estimate  Std. Error z value        Pr(>|z|)    
(Intercept)           306.181815  129.855953   2.358         0.01838 *  
alcohol                 0.808770    0.161800   4.999 0.0000005775408 ***
fixed.acidity           0.327392    0.150303   2.178         0.02939 *  
volatile.acidity       -2.716251    0.972265  -2.794         0.00521 ** 
citric.acid             1.257036    1.023627   1.228         0.21944    
chlorides              -8.957779    3.761275  -2.382         0.01724 *  
total.sulfur.dioxide   -0.011156    0.005186  -2.151         0.03145 *  
pH                      0.535113    1.22662

Now that we can see a model with all of the variables included, we can take some of these variables out one by one to get to something more reasonable.

Variables that appear that they can be omitted, in order or least statistical significance:<br>
free.sulfur.dioxide<br>
pH<br>
citric.acid<br>

We will remove one at a time to see whether the statistical significance of the other variables changes.

In [71]:
mod4 = glm(good_binary ~ alcohol + fixed.acidity + volatile.acidity + citric.acid + chlorides + total.sulfur.dioxide + residual.sugar + pH + density + sulphates, data=train, family="binomial")
summary(mod4)


Call:
glm(formula = good_binary ~ alcohol + fixed.acidity + volatile.acidity + 
    citric.acid + chlorides + total.sulfur.dioxide + residual.sugar + 
    pH + density + sulphates, family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.2104  -0.4119  -0.1843  -0.0971   3.1142  

Coefficients:
                        Estimate  Std. Error z value        Pr(>|z|)    
(Intercept)           305.052166  129.906417   2.348         0.01886 *  
alcohol                 0.811126    0.161721   5.016 0.0000005287079 ***
fixed.acidity           0.323963    0.149903   2.161         0.03068 *  
volatile.acidity       -2.681170    0.968805  -2.768         0.00565 ** 
citric.acid             1.278796    1.022129   1.251         0.21089    
chlorides              -9.014310    3.753290  -2.402         0.01632 *  
total.sulfur.dioxide   -0.012394    0.003973  -3.120         0.00181 ** 
residual.sugar          0.238863    0.090288   2.646         0.00816 *

In [73]:
mod5 = glm(good_binary ~ alcohol + fixed.acidity + citric.acid + volatile.acidity + chlorides + total.sulfur.dioxide + residual.sugar + density + sulphates, data=train, family="binomial")
summary(mod5)


Call:
glm(formula = good_binary ~ alcohol + fixed.acidity + citric.acid + 
    volatile.acidity + chlorides + total.sulfur.dioxide + residual.sugar + 
    density + sulphates, family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.1859  -0.4145  -0.1855  -0.0989   3.1189  

Coefficients:
                        Estimate  Std. Error z value        Pr(>|z|)    
(Intercept)           278.949042  111.708941   2.497         0.01252 *  
alcohol                 0.840763    0.143404   5.863 0.0000000045480 ***
fixed.acidity           0.283886    0.110396   2.572         0.01013 *  
citric.acid             1.210062    1.008988   1.199         0.23042    
volatile.acidity       -2.715590    0.968940  -2.803         0.00507 ** 
chlorides              -9.249074    3.713942  -2.490         0.01276 *  
total.sulfur.dioxide   -0.012559    0.003948  -3.181         0.00147 ** 
residual.sugar          0.227475    0.085680   2.655         0.00793 ** 
de

In [74]:
mod6 = glm(good_binary ~ alcohol + fixed.acidity + volatile.acidity + chlorides + total.sulfur.dioxide + residual.sugar + density + sulphates, data=train, family="binomial")
summary(mod6)


Call:
glm(formula = good_binary ~ alcohol + fixed.acidity + volatile.acidity + 
    chlorides + total.sulfur.dioxide + residual.sugar + density + 
    sulphates, family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.3008  -0.4118  -0.1884  -0.0992   3.1402  

Coefficients:
                        Estimate  Std. Error z value        Pr(>|z|)    
(Intercept)           270.685039  111.522136   2.427        0.015216 *  
alcohol                 0.873406    0.140649   6.210 0.0000000005304 ***
fixed.acidity           0.346505    0.097831   3.542        0.000397 ***
volatile.acidity       -3.379918    0.807427  -4.186 0.0000283868008 ***
chlorides              -8.011812    3.586640  -2.234        0.025497 *  
total.sulfur.dioxide   -0.011977    0.003963  -3.022        0.002507 ** 
residual.sugar          0.241378    0.084484   2.857        0.004275 ** 
density              -286.999008  112.008635  -2.562        0.010398 *  
sulphates       

The above seems to be a pretty good model as we have achieved our lowest AIC set up to this point, and all of our remaining variables have a high level of statistical significance.

We can now use this model to make our training set predictions.

In [75]:
pred6 = predict(mod6, type="response")
table(train$good_binary, pred6 > 0.5)

   
    FALSE TRUE
  0   935   32
  1    90   62

In [76]:
# Calculating our training set accuracy
(935 + 62)/ (935 + 62 + 90 + 32)

[1] 0.8909741

The above statistics suggests we were 89% accurate on the training set.

Now we can see how well our model performed against the test set.

In [77]:
TestPrediction = predict(mod6, newdata=test, type="response")
table(test$good_binary, TestPrediction>=0.5)

   
    FALSE TRUE
  0   395   20
  1    47   18

In [78]:
# Calculating test set accuracy
(395 + 18) / (395 + 18 + 20 + 47)

[1] 0.8604167

In this case, we are 86% correct against the test set, which is a pretty good outcome.

In the case that we simply predicted that all wines would be bad, as that is the more common outcome, we would have also been correct 86% of the time, so our fitted model does not offer additional predictive power over the simple baseline method.

In [17]:
# Calculating baseline accuracy
(395 + 20) / (395 + 20 + 47 + 18)

[1] 0.8645833

It could be worthwhile at this point to explore other models that may increase or predictive power on the data.